In [ ]:
import os
import pathlib

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
CAT_DATASET = './CAT_DATASET'
dataset_dir = pathlib.Path(CAT_DATASET)

In [ ]:
img_height = 150
img_width = 150
epochs = 10
batch_size = 2048

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  validation_split=0.2)

In [ ]:
train_gen = train_datagen.flow_from_directory(CAT_DATASET,
                                              target_size=(img_width, img_height),
                                              batch_size=256,
                                              class_mode='categorical',
                                              subset='training')

In [ ]:
val_gen = train_datagen.flow_from_directory(CAT_DATASET,
                                              target_size=(img_width, img_height),
                                              batch_size=256,
                                              class_mode='categorical',
                                              subset='validation')

In [ ]:
num_class = train_gen.num_classes
class_names = list(train_gen.class_indices.keys())
print(class_names)

### Model

In [ ]:
model_base = keras.applications.inception_resnet_v2.InceptionResNetV2(weights='imagenet', 
                                                                      include_top=False, 
                                                                      input_shape=(img_width,img_height,3))

model = keras.models.Sequential()
for layer in model_base.layers:
    layer.trainable = False

model.add(model_base)
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(512, activation='relu'))
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dense(67, activation='softmax'))
model.summary()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(train_gen,
                    steps_per_epoch = train_gen.samples // batch_size,
                    validation_data = val_gen, 
                    validation_steps = val_gen.samples // batch_size,
                    epochs = epochs)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
model.save('./MODEL_AWAL')

In [ ]:
def preprocess_input(x):
    x /= 255.
    x -= 0.5
    x *= 2.
    return x

In [ ]:
from urllib.parse import urlparse
a = urlparse(cat)
file_name = os.path.basename(a.path)

cat = "https://cdn-2.tstatic.net/jogja/foto/bank/images/kucing-sphynx_20151213_161613.jpg"
cat_path = tf.keras.utils.get_file(f'./{file_name}', origin=cat, cache_dir='./')
img = keras.preprocessing.image.load_img(
    cat_path, target_size=(img_height, img_width)
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

img_array = preprocess_input(img_array)

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])
print(class_names[np.argmax(score)], round(100 * np.max(score)), "%")

In [ ]:
print(predictions)